In [1]:
from datasets import load_dataset, DatasetDict, load_from_disk
from transformers import AutoTokenizer
dataset = load_from_disk('./')

In [ ]:
for i in range(10):
    print(dataset['train'][i])

In [1]:
from datasets import load_dataset, DatasetDict, load_from_disk
from transformers import AutoTokenizer

# Load WikiText-103 dataset
dataset = load_dataset("wikitext", "wikitext-103-raw-v1", split="train")

dataset = load_from_disk("../processed_datadir/wikitext-103-raw-train")

Found cached dataset wikitext (/home/songx_lab/cse12012530/.cache/huggingface/datasets/wikitext/wikitext-103-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126)


In [ ]:
# read next fast speed
# dataset.save_to_disk("wikitext-103-raw-train")

In [2]:
from transformers import BertTokenizerFast
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")
print(tokenizer.vocab_size)
print("Tokenizer max length: ", tokenizer.model_max_length)  # change your own model max input length
tokenizer.model_max_length = 128
print("Tokenizer max length after change: ", tokenizer.model_max_length) 

# Some tokenizer have a pad_token, which is used to pad a sequence up to max_length. But GPT2 tokenizer doesn't have it

# convenient function to load tokenizer next time
tokenizer.save_pretrained(f"../tokenizer_save/tokenizer-{tokenizer.name_or_path}-{tokenizer.model_max_length}")

30522
Tokenizer max length:  512
Tokenizer max length after change:  128


('../tokenizer_save/tokenizer-bert-base-uncased-128/tokenizer_config.json',
 '../tokenizer_save/tokenizer-bert-base-uncased-128/special_tokens_map.json',
 '../tokenizer_save/tokenizer-bert-base-uncased-128/vocab.txt',
 '../tokenizer_save/tokenizer-bert-base-uncased-128/added_tokens.json',
 '../tokenizer_save/tokenizer-bert-base-uncased-128/tokenizer.json')

In [ ]:
print(tokenizer)

BertTokenizerFast(name_or_path='bert-base-uncased', vocab_size=30522, model_max_length=256, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})


In [22]:
print(tokenizer.pad_token_id)
print(tokenizer.sep_token_id)

0
102


In [23]:
print(tokenizer.pad_token_type_id)

0


In [ ]:
for i in range(10):
    print(dataset[i])

In [35]:
tokenized_inputs = tokenizer("a " * 10000,   # 果然给你拆分了，免得浪费文本
        max_length=tokenizer.model_max_length, return_overflowing_tokens=True,return_length=True)

In [ ]:
print(tokenized_inputs)

In [10]:
import multiprocessing

num_proc = multiprocessing.cpu_count()
print(f"The max length for the tokenizer is: {tokenizer.model_max_length}")
# 害怕太多pad，导致模型只预测pad，所以只取最大长度的文本
def group_texts(examples):
    tokenized_inputs = tokenizer(examples["text"], truncation=True,   # 在这就不带文本了
        max_length=tokenizer.model_max_length, return_overflowing_tokens=True,return_length=True)
    input_ids_list = []
    token_type_ids_list = []
    attention_mask_list = []
    
    for length, input_ids in zip(tokenized_inputs["length"], tokenized_inputs["input_ids"]):
        if length == tokenizer.model_max_length:
            input_ids_list.append(input_ids)
        # else:
        #     input_ids_list.append(input_ids + [tokenizer.pad_token_id] * (tokenizer.model_max_length - length))
    for length, token_type_ids in zip(tokenized_inputs["length"], tokenized_inputs["token_type_ids"]):
        if length == tokenizer.model_max_length:
            token_type_ids_list.append(token_type_ids)
        # else:
        #     token_type_ids_list.append(token_type_ids + [tokenizer.pad_token_type_id] * (tokenizer.model_max_length - length))
    for length, attention_mask in zip(tokenized_inputs["length"], tokenized_inputs["attention_mask"]):
        if length == tokenizer.model_max_length:
            attention_mask_list.append(attention_mask)
        # else:
        #     attention_mask_list.append(attention_mask + [0] * (tokenizer.model_max_length - length))
        
    return {"input_ids": input_ids_list, "token_type_ids": token_type_ids_list, "attention_mask": attention_mask_list}

# preprocess dataset
tokenized_datasets = dataset.map(group_texts, batched=True, remove_columns=["text"], num_proc=num_proc)
tokenized_datasets.features



The max length for the tokenizer is: 128


Map (num_proc=16):   0%|          | 0/1801350 [00:00<?, ? examples/s]

{'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'token_type_ids': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}

In [11]:
print(len(tokenized_datasets))

501627


In [12]:
print(len(dataset))

1801350


In [ ]:
for i in range(10):
    print(tokenized_datasets[i])

In [ ]:
for i in range(10):
    # print(tokenizer.decode(tokenized_datasets[i]["input_ids"]))
    print(tokenized_datasets[i])
    print("---------------------------------------------------------")

In [3]:
# from datasets import load_dataset, DatasetDict, load_from_disk
# from transformers import AutoTokenizer

# tokenized_datasets = load_from_disk("wikitext-103-raw-train-bert-half-processed")

In [13]:
splits = tokenized_datasets.train_test_split(test_size=0.05, shuffle=True)

In [29]:
preprocessed_splits = DatasetDict({
    "train": splits["train"],
    "validation": splits["test"],
    "test": load_dataset("wikitext", "wikitext-103-raw-v1", split="test")
})

Found cached dataset wikitext (/home/songx_lab/cse12012530/.cache/huggingface/datasets/wikitext/wikitext-103-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126)


### Then we can construct our own wikitext test data
The biggest difference is that train and evaluate always stay with CrossEntry Loss, and won't care the real text effect, but text should have the overflowing tokens for GPT2
 

In [30]:

preprocessed_splits["test"] = preprocessed_splits["test"].map(group_texts,batched=True, remove_columns=["text"],num_proc=4)

Map (num_proc=4):   0%|          | 0/4358 [00:00<?, ? examples/s]

In [20]:
print(preprocessed_splits["test"].features)

{'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None), 'token_type_ids': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None), 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None), 'overflow_to_sample_mapping': Value(dtype='int64', id=None)}


In [31]:
for i in range(10):
    print(preprocessed_splits["test"][i])

{'input_ids': [101, 2728, 8945, 11314, 2121, 2003, 2019, 2394, 2143, 1010, 2547, 1998, 3004, 3364, 1012, 2002, 2018, 1037, 4113, 1030, 1011, 1030, 4626, 2535, 2006, 1996, 2547, 2186, 1996, 3021, 1999, 2456, 1012, 2023, 2001, 2628, 2011, 1037, 4626, 2535, 1999, 1996, 2377, 22914, 2015, 2517, 2011, 4079, 15037, 1010, 2029, 2001, 2864, 1999, 2541, 2012, 1996, 2548, 2457, 3004, 1012, 2002, 2018, 1037, 4113, 2535, 1999, 1996, 2547, 2186, 3648, 2198, 15046, 1999, 2526, 1012, 1999, 2432, 8945, 11314, 2121, 5565, 1037, 2535, 2004, 1000, 7010, 1000, 1999, 1996, 2792, 1000, 11389, 1005, 1055, 2466, 1000, 1997, 1996, 2547, 2186, 1996, 2146, 3813, 1025, 2002, 5652, 4077, 5889, 2928, 2844, 1998, 7256, 6213, 2072, 1012, 2002, 2001, 3459, 1999, 1996, 2384, 3004, 5453, 1997, 1996, 5170, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [32]:
print(len(preprocessed_splits["test"]))

1157


In [37]:
try:
    preprocessed_splits["test"] = preprocessed_splits["test"].remove_columns(["overflowing_tokens_ids"])
except:
    pass
try:
    preprocessed_splits["test"] = preprocessed_splits["test"].remove_columns(["num_truncated_tokens"])
except:
    pass

In [38]:
preprocessed_splits.save_to_disk("../processed_datadir/wikitext-103-preprocessed-ws-notext-bert-128-wtest")

Flattening the indices:   0%|          | 0/476545 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/476545 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/25082 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/25082 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1157 [00:00<?, ? examples/s]